### classify stackoverflow posts using tf-idf features, using one-vs-all SVM

- note: use the same tokenizer rules as Keras

In [14]:
import csv
import os
import re
import sys
import numpy as np
import tensorflow as tf

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Dropout, Input, Flatten, Activation, Lambda
from keras.layers import Conv1D, MaxPooling1D, Embedding, GlobalAvgPool1D
from keras.models import Model, Sequential

from sklearn import preprocessing, svm
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
from sklearn.model_selection import ParameterGrid
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.pipeline import Pipeline

# my stuff in the helpers/ directory
from helpers import files_helper, texts_helper, metrics_helper, tags_helper

In [2]:
SEED=42

In [3]:
np.random.seed(seed=SEED)

In [4]:
texts, labels = files_helper.read_stackoverflow_sample_small()

In [5]:
MAX_NB_WORDS = 20000
MAX_SEQUENCE_LENGTH = 1000
VALIDATION_SPLIT = 0.2
LABELS_MIN_DOC_COUNT = int(10)

In [6]:
tokenizer = Tokenizer(num_words=MAX_NB_WORDS,
                     filters='\'!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index
inverse_word_index = texts_helper.build_inverse_word_index(word_index)

In [7]:
[inverse_word_index[idx] for idx in sequences[0]]

['apply',
 'onclick',
 'event',
 'to',
 'an',
 'option',
 'i',
 'am',
 'using',
 'zend',
 'form',
 'to',
 'create',
 'a',
 'form',
 'i',
 'am',
 'also',
 'using',
 'for',
 'javascript',
 'this',
 'gt',
 'radio',
 'alone',
 'array',
 'label',
 'gt',
 'are',
 'you',
 'going',
 'to',
 'be',
 'taking',
 'part',
 'with',
 'anyone',
 'else',
 'required',
 'gt',
 'true',
 'onclick',
 'gt',
 'gt',
 'array',
 'yes',
 'gt',
 'yes',
 'no',
 'gt',
 'no',
 'at',
 'the',
 'moment',
 'the',
 'onclick',
 'event',
 'works',
 'if',
 'any',
 'option',
 'is',
 'selected',
 'how',
 'do',
 'i',
 'get',
 'it',
 'to',
 'work',
 'for',
 'just',
 'yes',
 'being',
 'selected']

In [8]:
truncated_labels = tags_helper.truncate_labels(labels,LABELS_MIN_DOC_COUNT)

In [9]:
lb = preprocessing.MultiLabelBinarizer()
binary_labels = lb.fit_transform(truncated_labels)

In [10]:
# tag position => fraction of docs having that tag
tag_probabilities_index = tags_helper.get_probabilities_index(binary_labels)

In [11]:
tokenized_texts = []

for seq in sequences:
    tokenized_txt = " ".join([inverse_word_index[idx] for idx in seq])
    tokenized_texts.append(tokenized_txt)

In [12]:
data = tokenized_texts
indices = np.arange(len(data))
np.random.shuffle(indices)

data = [data[i] for i in indices]
labels_1 = binary_labels[indices]
num_validation_samples = int(VALIDATION_SPLIT * len(data))

X_train = data[:-num_validation_samples]
Y_train = labels_1[:-num_validation_samples]
X_val = data[-num_validation_samples:]
Y_val = labels_1[-num_validation_samples:]

len(data),labels_1.shape

(40000, (40000, 1469))

In [16]:
clf = OneVsRestClassifier(Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', svm.LinearSVC()),
]), n_jobs=-1)

parameters = [
    { 
          "estimator__clf__penalty": ["l1","l2"],
          "estimator__clf__dual":[False,True],
          "estimator__clf__multi_class":["crammer_singer","ovr"],
          "estimator__clf__tol": [0.01,0.001],
          "estimator__vect__min_df": [5,10],
          "estimator__vect__max_features": [MAX_NB_WORDS]  
    }
]

In [17]:
best_score = float("-inf")

for g in ParameterGrid(parameters):
    clf.set_params(**g)
    clf.fit(X_train,Y_train)
    
    Y_pred = clf.predict(X_val)
    
    current_score = f1_score(Y_val,Y_pred,average='micro')
    
    print("current_score was {} and the current grid was {}".format(current_score,g))
    
    if current_score > best_score:
        best_score = current_score
        best_grid = g

current_score was 0.47598982927715217 and the current grid was {'estimator__clf__tol': 0.01, 'estimator__clf__dual': False, 'estimator__clf__penalty': 'l1', 'estimator__clf__multi_class': 'crammer_singer', 'estimator__vect__max_features': 20000, 'estimator__vect__min_df': 5}


KeyboardInterrupt: 